In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tsmoothie.smoother import GaussianSmoother
import spikeinterface
import spikeinterface.full as si
import spikeinterface.extractors as se                                      
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
import spikeinterface.postprocessing as sp
import spikeinterface.preprocessing as spre
import spikeinterface.qualitymetrics as qm
import helper_functions as helper
from spikeinterface.sorters import run_sorter_local

In [ ]:
full_path = '/mnt/disk15tb/mmpatil/Axontracking/000026/data.raw.h5'
rec_id = 0
common_el = list(range(30000))
while rec_id < 3: #True:
    rec_name = 'rec' + '%0*d' % (4, rec_id)
    try:   
        rec = si.MaxwellRecordingExtractor(full_path,stream_name='well000',rec_name=rec_name)
        rec_el = rec.get_property("contact_vector")["electrode"]
        common_el = list(set(common_el).intersection(rec_el))
        rec_id += 1
    except Exception:
        break
            

In [ ]:
rec_id


In [ ]:
rec_list = []
for r in range(rec_id): #rec_id
    rec_name = 'rec' + '%0*d' % (4, r)
    rec = si.MaxwellRecordingExtractor(full_path,stream_name='well000',rec_name=rec_name)
    ch_id = rec.get_property("contact_vector")['device_channel_indices']
    rec_el = rec.get_property("contact_vector")["electrode"]
    
    chan_idx = [np.where(rec_el == el)[0][0] for el in common_el]
    print(f'x{rec.get_property("contact_vector")["x"][chan_idx]}y{rec.get_property("contact_vector")["y"][chan_idx]}')
    sel_channels = rec.get_channel_ids()[chan_idx]
    rec_centered = si.filter(rec, band=[150,4999])
    rec_list.append(rec_centered.channel_slice(sel_channels, renamed_channel_ids=list(range(len(chan_idx)))))

multirecording = si.concatenate_recordings(rec_list)

In [ ]:
full_path = '/mnt/disk20tb/PrimaryNeuronData/Maxtwo/Folic_Acid_T2/Folic_Acid_T2/240123/M07039/ActivityScan/000003/data.raw.h5'
import h5py
stream_id = "well000"
h5 = h5py.File(full_path, mode="r")
rec_names = list(h5["wells"][stream_id].keys())
recs =list()
for rec_name in rec_names:
    recs.append(si.MaxwellRecordingExtractor(full_path,stream_name=stream_id,rec_name=rec_name))

aggregated_rec = si.aggregate_channels(recs)
print(aggregated_rec)

In [ ]:
chan_idx

In [ ]:
multirecording.get_channel_ids()

In [ ]:
#Reading the file, BP filtering
#path of the network file
local_path= '/mnt/disk15tb/mmpatil/Axontracking/000026/data.raw.h5' #network data from chip 16848
recordings =list()
for recnames in ['rec0000', 'rec0001', 'rec0002', 'rec0003', 'rec0004', 'rec0005', 'rec0006', 'rec0007', 'rec0008', 'rec0009', 'rec0010', 'rec0011', 'rec0012', 'rec0013', 'rec0014', 'rec0015', 'rec0016', 'rec0017', 'rec0018', 'rec0019', 'rec0020', 'rec0021', 'rec0022', 'rec0023', 'rec0024', 'rec0025', 'rec0026', 'rec0027', 'rec0028', 'rec0029', 'rec0030', 'rec0031', 'rec0032', 'rec0033', 'rec0034', 'rec0035', 'rec0036', 'rec0037', 'rec0038', 'rec0039']:
    recordings.append(se.read_maxwell(local_path,stream_id='well000',rec_name=recnames)) #well000 - well005 for maxtwo 

recording1 = si.aggregate_channels(recordings)

#recording = si.ConcatenateSegmentRecording([recording1,recording2])
channel_ids = recording1.get_channel_ids()
fs = recording1.get_sampling_frequency()
num_chan = recording1.get_num_channels()
num_seg = recording1.get_num_segments()
total_recording = recording1.get_total_duration()

#print('Channel ids:', channel_ids)
print('Sampling frequency:', fs)
print('Number of channels:', num_chan)
print('Number of segments:', num_seg)
print(f"total_recording: {total_recording} s")

recording_bp = spre.bandpass_filter(recording1, freq_min=300, freq_max=3000)

recodring_cmr = spre.common_reference(recording_bp, reference='global', operator='median')
#recording_chunk = recodring_cmr.frame_slice(start_frame= 1*fs,end_frame=425*fs)
recording_chunk = recodring_cmr.frame_slice(start_frame= 0*fs,end_frame=180*fs)
print(f"chunk duration: {recording_chunk.get_total_duration()} s")

In [3]:
local_path= '/mnt/disk20tb/Organoids/Maxone/RTT_Organoids_T2_02022024_KMB/RTT_Organoids_T2_02022024_KMB/240222/19401/Network/000051/data.raw.h5' #network data from chip 16848


recording1 = se.read_maxwell(local_path,stream_id='well000')

#recording = si.ConcatenateSegmentRecording([recording1,recording2])
channel_ids = recording1.get_channel_ids()
fs = recording1.get_sampling_frequency()
num_chan = recording1.get_num_channels()
num_seg = recording1.get_num_segments()
total_recording = recording1.get_total_duration()

#print('Channel ids:', channel_ids)
print('Sampling frequency:', fs)
print('Number of channels:', num_chan)
print('Number of segments:', num_seg)
print(f"total_recording: {total_recording} s")

recording_bp = spre.bandpass_filter(recording1, freq_min=300, freq_max=3000)

recodring_cmr = spre.common_reference(recording_bp, reference='global', operator='median')
#recording_chunk = recodring_cmr.frame_slice(start_frame= 1*fs,end_frame=425*fs)
recording_chunk = recodring_cmr.frame_slice(start_frame= 0*fs,end_frame=300*fs)
print(f"chunk duration: {recording_chunk.get_total_duration()} s")

Sampling frequency: 20000.0
Number of channels: 988
Number of segments: 1
total_recording: 300.08 s
chunk duration: 300.0 s


## RUnnig sorting

In [ ]:
traces = recording_chunk.get_traces(start_frame=100, end_frame=50000*15, segment_index=0,return_scaled=True)


In [ ]:
traces = recording_chunk.get_traces(start_frame=290500, end_frame=291800, segment_index=0,return_scaled=True)
plt.figure(figsize=(12,2))
plt.plot(traces[:,500])
plt.savefig('/home/mmp/Documents/traces/fulltrace.svg',format='svg')
plt.show()

In [ ]:
traces = recording_chunk.get_traces(start_frame=278860, end_frame=278910, segment_index=0,return_scaled=True)
plt.figure(figsize=(3,4))
plt.plot(traces[:,500])
plt.savefig('/home/mmp/Documents/traces/zoomedin.svg',format='svg')
plt.show()

In [5]:
#aggregated_rec= aggregated_rec.save(fodler="./sorting/recordingtest",progress_bar=True,verbose=True)
default_KS2_params = ss.get_default_sorter_params('kilosort2')
print(default_KS2_params)
#default_KS2_params['keep_good_only'] = True
# default_KS2_params['detect_threshold'] = 12
# default_KS2_params['projection_threshold']=[18, 10]
default_KS2_params['n_jobs'] = 32
# global_job_kwargs = dict(n_jobs=32, total_memory="8G", progress_bar=True)
# si.set_global_job_kwargs(**global_job_kwargs)
run_sorter = ss.run_sorter('kilosort2',recording=recording1, output_folder="./sorting/KMB_802F",docker_image= "rohanmalige/benshalom:v1",verbose=True, **default_KS2_params)

## if running on NERSC:
#run_sorter_local("kilosort2",recording_chunk, output_folder="./sorting/FolicAcid10mg", delete_output_folder=False,verbose=True,with_output=True,**default_KS2_params)
#run_sorter = ss.run_sorter('kilosort2',recording= recording_chunk, output_folder="/mnt/disk15tb/mmpatil/Spikesorting/sorter_output/kilosort2",docker_image= True,verbose=True, **default_KS2_params)

{'detect_threshold': 6, 'projection_threshold': [10, 4], 'preclust_threshold': 8, 'momentum': [20.0, 400.0], 'car': True, 'minFR': 0.1, 'minfr_goodchannels': 0.1, 'freq_min': 150, 'sigmaMask': 30, 'lam': 10.0, 'nPCs': 3, 'ntbuff': 64, 'nfilt_factor': 4, 'NT': None, 'AUCsplit': 0.9, 'wave_length': 61, 'keep_good_only': False, 'skip_kilosort_preprocessing': False, 'scaleproc': None, 'save_rez_to_mat': False, 'delete_tmp_files': ('matlab_files',), 'delete_recording_dat': False, 'n_jobs': 48, 'chunk_duration': '1s', 'progress_bar': True, 'mp_context': None, 'max_threads_per_process': 1}
installation_mode='auto' switching to installation_mode: 'github'


AssertionError: The container requires a NVIDIA GPU capability, but it is not available

In [ ]:

# loading the KS2 sorted object
sortingKS3 = run_sorter.remove_empty_units()
sortingKS3 = spikeinterface.curation.remove_excess_spikes(sortingKS3,multirecording) #Sometimes KS returns spikes outside the number of samples. < https://github.com/SpikeInterface/spikeinterface/pull/1378>

sortingKS3= sortingKS3.save(folder = './sorting/test_ks2',overwrite=True)
#sorting_KS3 = s.Kilosort2Sorter._get_result_from_folder('./sorting/FolicAcidT2M07038_2_KS/sorter_output')
total_units = sortingKS3.get_unit_ids()
print(len(total_units))
#print(len(total_units))
channel_ids = recording_chunk.get_channel_ids()
print(channel_ids)

## Extracting the waveforms

In [ ]:

job_kwargs = dict(n_jobs=32, chunk_duration="1s", progress_bar=True)
#waveforms = si.extract_waveforms(recording_chunk,sorting_KS3,folder="./waveformsblock1_7min",overwrite=True, ms_before=1., ms_after=2.,**job_kwargs)
waveforms = si.extract_waveforms(multirecording,sortingKS3,folder='./sorting/test_waveforms/',overwrite=True,**job_kwargs)
print(waveforms)

In [ ]:
si.export_to_phy(waveform_extractor=waveforms,output_folder='./sorting/SPTAN1_1_16793_phy',**job_kwargs)

In [ ]:
extremum_channels_ids =si.get_template_extremum_channel(waveforms,peak_sign ='both',mode='at_index')

In [ ]:
template = waveforms.get_template(5,mode='median')
template.shape

In [ ]:
waveforms = si.load_waveforms('/mnt/disk15tb/mmpatil/MEA_Analysis/IPNAnalysis/sorting/FolicAcidT2M07038_2_waveforms')
print(waveforms)
job_kwargs = dict(n_jobs=16, chunk_duration="1s", progress_bar=True)
sp.compute_spike_amplitudes(waveforms,load_if_exists=True,**job_kwargs)
metrics = qm.compute_quality_metrics(waveforms,load_if_exists=False,**job_kwargs)

In [ ]:
from spikeinterface.postprocessing import compute_spike_amplitudes, compute_principal_components
from spikeinterface.exporters import export_to_phy


_ = compute_principal_components(waveform_extractor=waveforms, n_components=3, mode='by_channel_global',**job_kwargs)

# the export process is fast because everything is pre-computed


In [ ]:
job_kwargs = dict(n_jobs=48, chunk_duration="1s", progress_bar=True)
sparsity = spikeinterface.core.compute_sparsity(waveforms ,method="radius", radius_um=40)
export_to_phy(waveform_extractor=waveforms, output_folder='/mnt/disk15tb/mmpatil/MEA_Analysis/IPNAnalysis/sorting/M07038_phy',sparsity=sparsity,**job_kwargs)

In [ ]:
display(update_qual_metrics)

In [ ]:
print(non_violated_units)

In [ ]:
len(non_violated_units)

In [ ]:
waveform_good = waveforms.select_units(non_violated_units,new_folder='./sorting/FolicAcidT2M07038_2_waveformsgood/')

In [ ]:
correlograms, bins = sp.compute_correlograms(waveform_good)

In [ ]:
correlograms.shape

In [ ]:
import matplotlib.pyplot as plt

# Extracting the correlation values between units 2 and 5 across all bins
# Adjusting indices for 0-based Python indexing
correlations = correlograms[2, 2, :]

# Plotting the histogram
plt.figure(figsize=(8, 6))
plt.hist(correlations, bins=20, color='blue', edgecolor='black')
plt.title('Histogram of Correlations Between Units 2 and 5 Across Bins')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Frequency')
plt.show()

In [ ]:
bins

In [ ]:
metrics = qm.compute_quality_metrics(waveform_good,load_if_exists=False,**job_kwargs)

In [ ]:
display(metrics)

In [ ]:
waveform_good = None

In [ ]:
waveform_good = si.load_waveforms('./sorting/waveformsgood_Folicacid10mg/')

In [ ]:
metrics = qm.compute_quality_metrics(waveform_good,load_if_exists=False,**job_kwargs)

In [ ]:
display(metrics)

In [ ]:

#locations = sp.compute_unit_locations(waveforms)
locations = sp.compute_unit_locations(waveforms)
print(type(locations))

#np.savetxt("unitloc_10mins.txt",locations)
fig,ax = plt.subplots(figsize=(10.5,6.5))
#sw.plot_probe_map(mu,ax=ax,with_channel_ids=False)
ax.set_facecolor('#000000') 
for x,y,z in locations:
    ax.scatter(x,y, s=5,c='deepskyblue')
ax.invert_yaxis()
plt.savefig(f'/home/mmp/Documents/multirecordingssort/axontrackingsort.svg',format='svg')